<a href="https://colab.research.google.com/github/Nicol295/Nicol295/blob/main/Copy_of_Conceptual_Database_Models_(Python_SQLAlchemy).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# models.py - Defining the structure of our data entities

from datetime import datetime
from sqlalchemy import create_engine, Column, Integer, String, Float, Boolean, Text, DateTime, JSON, ForeignKey
from sqlalchemy.orm import sessionmaker, declarative_base, relationship

# --- Database Setup (Conceptual) ---
# In a real app, this would be configured via Flask-SQLAlchemy or a similar ORM setup
# This is a simplified example for demonstration purposes.
DATABASE_URL = "sqlite:///makueni_farm_connect.db" # Example using SQLite
engine = create_engine(DATABASE_URL)
Session = sessionmaker(bind=engine)
Base = declarative_base() # Base class for our declarative models

# --- User Model ---
# Represents both farmers and buyers
class User(Base):
    __tablename__ = 'users' # Name of the database table
    id = Column(Integer, primary_key=True)
    username = Column(String(80), unique=True, nullable=False)
    password_hash = Column(String(128), nullable=False) # Stores hashed passwords for security
    user_type = Column(String(20), nullable=False) # 'farmer', 'buyer', 'admin'
    phone_number = Column(String(20), unique=True, nullable=False)
    location = Column(String(100)) # e.g., 'Wote', 'Kibwezi'
    date_joined = Column(DateTime, default=datetime.utcnow)

    # Relationships: One user can have many listings (if a farmer), and many orders (if a buyer)
    listings = relationship('ProductListing', backref='farmer', lazy=True, cascade="all, delete-orphan")
    orders_as_buyer = relationship('Order', foreign_keys='Order.buyer_id', backref='buyer', lazy=True, cascade="all, delete-orphan")

    def __repr__(self):
        return f"<User {self.username} ({self.user_type})>"

# --- Product Listing Model ---
# Represents a single item of produce listed for sale
class ProductListing(Base):
    __tablename__ = 'product_listings'
    id = Column(Integer, primary_key=True)
    farmer_id = Column(Integer, ForeignKey('users.id'), nullable=False) # Links to the farmer who posted it
    product_name = Column(String(100), nullable=False)
    product_category = Column(String(50)) # e.g., 'Fruits', 'Vegetables', 'Grains', 'Livestock'
    description = Column(Text)
    quantity = Column(Float, nullable=False) # e.g., 100
    unit = Column(String(20)) # e.g., 'kg', 'piece', 'bunch', 'head'
    price_per_unit = Column(Float, nullable=False) # Price per specified unit
    image_url = Column(String(255)) # Optional: URL to an uploaded product image
    is_available = Column(Boolean, default=True) # Flag to show if the listing is active
    date_posted = Column(DateTime, default=datetime.utcnow)

    # Flexible properties to accommodate diverse produce attributes
    # e.g., {'variety': 'Sukuma Wiki', 'color': 'green'} or {'breed': 'Dairy Cow', 'age': '2 years'}
    properties = Column(JSON)

    # Relationship: One listing can have many orders
    orders = relationship('Order', foreign_keys='Order.listing_id', backref='listing', lazy=True, cascade="all, delete-orphan")

    def __repr__(self):
        return f"<Listing {self.product_name} by {self.farmer.username}>"

# --- Order Model ---
# Represents a transaction where a buyer places an order for a product listing
class Order(Base):
    __tablename__ = 'orders'
    id = Column(Integer, primary_key=True)
    listing_id = Column(Integer, ForeignKey('product_listings.id'), nullable=False) # Links to the ordered product listing
    buyer_id = Column(Integer, ForeignKey('users.id'), nullable=False) # Links to the buyer who placed the order
    ordered_quantity = Column(Float, nullable=False) # Quantity the buyer wants
    total_price = Column(Float, nullable=False) # Calculated total price for the order
    order_status = Column(String(20), default='pending') # 'pending', 'accepted', 'rejected', 'completed'
    order_date = Column(DateTime, default=datetime.utcnow)

    def __repr__(self):
        return f"<Order {self.id} for {self.listing.product_name} by {self.buyer.username}>"

# --- Database Initialization (For initial setup) ---
# Uncomment and run this once to create the tables in your database
# Base.metadata.create_all(engine)